# Chapter 9 텍스트를 위한 인공신경망

## 9-1 순차 데이터와 순환 신경망

### 1) 순차 데이터 (sequential data)

   * text data : 단어의 순서가 중요한 데이터

            I am a student,      
            Am student a I,       
            student a am I,......

   * 시계열데이터(time series data)

          그제, 어제, 오늘의 주가지수
          계절별 실업률
          분기별 경제성장률, ......



순차 데이터의 핵심은 "기억", 즉 앞에 나온 과거 일에 대한 기억이다. 예컨대 <u>분류가 목적인 Fashion Mnist데이터</u> 분석에서 데이터의 앞뒤 순서는 중요하지않다. 우리는 그 데이터를 분석하기 위해 dense층, conv2D층을 이용했다.

이처럼 앞서 들어온 데이터를 다시 돌아볼 필요가 없는 신경망을 **feedforward neural network**이라고 부른다. 데이터의 흐름이 그냥 앞으로만 진행된다는 의미인데 우리가 7장에서 공부한 밀집층(완전연결신경망)과 8장에서 공부한 합성곱신경망은 모두 피드포워드신경망에 속한다.

### 2) 순환신경망(RNN, recurrent neural network)

* **timestep** : 이전 샘플에 대한 기억을 가지고 샘플을 처리하는 한 단계.

  예를 들어 일주일치 주가지수 데이터를 순차데이터로 본다면 5개의 타임스텝이 있다. 오늘(시점 t)의 주가지수를 처리해서 출력(가령 예측)을 만들 때에는 오늘 데이터 뿐아니라 어제(시점 t-1)의 데이터를 분석한 출력도 함께 이용한다. 그런데 어제 얻은 출력은 어제데이터와 그 전날(시점 t-2)의 데이터도 이용한 것이다. 이런 식으로 본다면 오늘 얻은 출력에는 시점 t, t-1, t-2, ...의 결과들이 영향을 미친다. 물론 과거의 데이터일수록 영향력은 작아질 것이다. 이처럼 <u>순환신경망에서는 과거데이터에 대한 기억을 이용한다.</u>
  
  텍스트를 예로 든다면 단어나 알파벳 문자가 타임스텝이 된다. 가령
  
        "Busan is ready for the World EXPO 2030"
        
  라는 문장을 단어로 된 순차데이터로 본다면 <u>8개의 타임스텝으로 이루어진 시퀀스데이터</u>가 된다.

* **cell** : 순환신경망에서 층을 일컫는 용어

  489쪽 맨 위의 그림을 보자. 붉은색 고리만 없다면 왼쪽 층은 완전연결 신경망층이다. 즉 순환신경망 층은 완전연결 신경망층과 비슷하지만 각 뉴런에서 나온 출력이 다시 그 뉴런에 입력되는 순환이 일어난다는 점이 다르다. 이런 뉴런들이 모인 층을 순환신경망에서는 '셀'이라고 부른다.

  즉 셀에 새로운 입력이 들어오면 과거의 기억과 새로운 입력을 가지고 기억을 업데이트한다. 따라서 순환신경망 셀의 입력은 새로운 데이터, 그리고 과거 상태(이전의 기억) 두 가지라고 할 수 있다.


* **은닉상태(hidden state)** : 셀의 출력
  
  순환신경망의 층(셀)에서는 어떤 작업이 진행될까?
  
  완전연결층과 마찬가지로 입력데이터에 가중치를 곱하고 절편을 더한 결과를 활성화함수에 입력해서 출력을 만드는 것은 똑같다. 이때 <u>활성화함수로는 주로 hyperbolic tangent(tanh) 함수를 이용</u>(491쪽 그림 참조)하는데, 이 출력을 **"은닉 상태(hidden state)"**라고 부른다.
  
  그런데 <u>순환신경망에서 달라지는 것은 이 출력, 즉 은닉 상태를 그 다음 타임스텝에서 다시 이용한다는 점</u>이다. 따라서 **순환신경망 뉴런에서 필요한 가중치는 두 가지**가 된다. 하나는 그 타임스텝의 입력데이터에 곱해지는 가중치(교재에서는 $w_x$로 표현)이고 하나는 이전 타임스텝의 은닉상태에 곱해지는 가중치($w_b$)다.

  492쪽 그림을 잘 보자. 큰 동그라미는 셀을 나타내고 회색 작은 동그라미는 활성화함수다.

  최초의 은닉상태 $h_0$는 보통 0으로 초기화한다. 첫번째 타임스텝의 데이터에 가중치 $w_x$를 곱하고 활성화함수를 이용해서 은닉상태 $h_1$을 구한다.

  그 다음 타임스텝에서는 입력데이터에 $w_x$를 곱하고 $h_1$에는 $w_b$를 곱한 결과에 활성화함수를 이용해서 $h_2$를 얻는다. RNN에서는 활성화함수로 hyperbolic tangent(tanh) 함수(491쪽 그림 참조)를 기본으로 이용한다.

  $ h_1 = \tanh(w_x \cdot x_1 + w_b \cdot h_0 + b) $

  $ h_2 = \tanh(w_x \cdot x_2 + w_b \cdot h_1 + b) $

  $ \ldots $

각 타임스텝이 단어 하나로 이루어진다고 가정하면 위의 설명에서 $x_1 , x_2, \cdots $가 각각 단어 하나씩을 나타낸다. 그 단어들로 이루어진 문장을 하나의 시퀀스데이터로 본다면 $h_i $들은 문장을 이루는 단어의 수만큼 생길 것이다.

물론 컴퓨터가 알파벳이나 한글 문자로 된 단어를 그대로 받아들일 수는 없으므로 각 단어는 모두 숫자 배열로 표현된다. 모든 단어를 모은 어휘사전이 500개의 단어로 구성된다고 해보자. 그렇다면 각 단어는 500개 숫자 중 하나로 나타낼 수 있을 것이다. 그 단어를 one hot encoding으로 나타낸다면 500개 값으로 이루어진 배열에서 특정 위치에서만 1 값을 갖게 될 것이다. 즉 모든 단어는 길이 500인 1차원 배열로 표시된다.

"I am a student"라는 시퀀스가 있다고 하자. 네 단어로 이루어져있으므로 타임스텝의 수가 4다. 어휘사전이 500개 단어로 이루어져있다고 하자. 네 단어는 각각 길이 500인 1차원 배열이다. 타임스텝마다 입력층의 길이(feature의 수)는 500이다. 따라서 "I am a student"라는 하나의 시퀀스는 (4, 500) 2차원 배열이 된다. 당연히 입력데이터에 곱해지는 가중치 $w_x $의 길이는 500이다.

입력층과 연결되는 순환층은 여러 개의 뉴런으로 구성된다. 만일 뉴런의 수가 100개라고 해보자. 완전연결에 필요한 파라미터의 수는? $ 500 \times 100 = 50,000 $개다. 즉 파라미터의 수는 단어 사전의 크기와 뉴런의 수에 따라 결정된다. 그리고 뉴런 수만큼의 절편도 필요하다(100개).

게다가 RNN에서는 이어지는 타임스텝마다 모든 뉴런의 출력을 입력으로 받아들이므로 이때 필요한 파라미터는 $(뉴런의 수)^2 $다. 뉴런이 100개라면 1만개의 파라미터가 필요하게 된다.

즉 필요한 파라미터의 수는 $50,000+100+10,000 = 60,100$개다.

### 3) 셀의 가중치와 입출력

* 순환층의 가중치

  이제 493쪽 위의 그림(입력 4개, 순환층 뉴런 3개)을 가지고 가중치의 갯수를 알아보자. 입력층 $x_1 , x_2 , x_3 , x_4 $은 순환층의 뉴런 $r_1 , r_2 , r_3 $과 모두 연결되므로 12개의 가중치($w_x $)가 필요하다.

  한편, 3개의 뉴런으로 이루어진 순환층에서 이전 타임스텝의 결과를 재활용할 때에는 각 뉴런의 출력이 3개 모든 뉴런에 전달된다(493쪽 아래 그림). 따라서 필요한 가중치는 9개($w_b $)다. 또한 뉴런마다 절편이 하나씩 필요하므로 파라미터의 총 수는 12 + 9 + 3 = 24개다.

    - 퀴즈 : 입력이 5개, 뉴런이 4개인 순환층이 있다고 하자. 모델에 필요한 파라미터는 몇 개인가?

* 순환층의 입력과 출력 (교재 494-6쪽 설명을 잘 이해하려면 9-2절에서 공부할 IMDB 영화리뷰 데이터셋 분석을 먼저 살펴보는 것이 좋다.)
  

* 순환층이 여럿인 경우 (심층 RNN)

  순환층이 둘이라고 해보자(495쪽 그림). 앞에서와 마찬가지로 "I am a student"라는 시퀀스가 있고 어휘 사전은 500개 단어로 되어있다고 하자. 첫번째 순환층의 셀에 입력되는 배열은 (4, 500)배열이다. 두번째 순환층 셀에 입력되는 배열도 마찬가지로 2차원 배열이어야한다. 이를 위해서는 첫번째 층에는 `return_sequences = True`라는 옵션을 추가해야한다. 마지막 타임스텝의 출력만 내보내는 것이 아니라 모든 타임스텝의 출력을 두번째 층에 전달해야한다는 의미다.

  - 퀴즈 : 첫번째 순환층의 뉴런이 100개라고 해보자. 이 경우 두번째 순환층 셀 입력 배열의 shape은?

* RNN의 출력층 : 밀집층으로 샘플을 분류한다. 다중분류면 softmax, 이진분류면 sigmoid 활성화함수 이용.

  496쪽 그림을 보자. 입력 샘플 크기가 (20, 100)이고 순환층은 10개의 뉴런으로 구성된다.
  - 퀴즈 : time step의 길이는? feature의 수는? 뉴런 출력의 차원은? 길이는?

  - 퀴즈 : 마지막 dense 출력층 앞에 Flatten이 필요할까?



9-2절로 넘어가지 전에 498-9쪽의 마무리와 확인문제를 공부하자.